## Solutions for Turtlebot Project

<img src="../img/robotignite_logo_text.png"/>

## Index: 

* <a href="#SolutionStep1">Solution Step 1</a>
* <a href="#SolutionStep2">Solution Step 2</a>
* <a href="#SolutionStep3">Solution Step 3</a>
* <a href="#SolutionStep4">Solution Step 4</a>

## Solution Step 1: Read and Write Topics <p id="SolutionStep1"></p>

<table style="float:left;background: #407EAF">
<tr>
<th>
<p class="transparent">2. Create the Topic Publisher to move Turtlebot.</p>
</th>
</tr>
</table>

<p style="background:#3B8F10;color:white;" id="prg-2-1">**Python File: cmd_vel_publisher.py** </p>

In [ ]:
#! /usr/bin/env python

import rospy
from geometry_msgs.msg import Twist


class CmdVelPub(object):
    def __init__(self):
        self._cmd_vel_pub = rospy.Publisher('/cmd_vel', Twist, queue_size=1)
        self._twist_object = Twist()
        self.linearspeed = 0.3
        self.angularspeed = 0.5
        
    def move_robot(self, direction):
        if direction == "forwards":
            self._twist_object.linear.x = self.linearspeed
            self._twist_object.angular.z = 0.0
        elif direction == "right":
            self._twist_object.linear.x = 0.0
            self._twist_object.angular.z = -self.angularspeed
        elif direction == "left":
            self._twist_object.linear.x = 0.0
            self._twist_object.angular.z = self.angularspeed
        elif direction == "backwards":
            self._twist_object.linear.x = -self.linearspeed
            self._twist_object.angular.z = 0.0
        elif direction == "stop":
            self._twist_object.linear.x = 0.0
            self._twist_object.angular.z = 0.0
        else:
            pass
        
        self._cmd_vel_pub.publish(self._twist_object)


if __name__ == "__main__":
    rospy.init_node('cmd_vel_publisher_node')
    cmd_publisher_object = CmdVelPub()
    
    rate = rospy.Rate(1)
    
    ctrl_c = False
    def shutdownhook():
        # works better than the rospy.is_shut_down()
        global ctrl_c
        global twist_object
        global pub
        
        rospy.loginfo("shutdown time!")
        
        ctrl_c = True
        cmd_publisher_object.move_robot(direction="stop")
    
    rospy.on_shutdown(shutdownhook)
    
    while not ctrl_c:
        cmd_publisher_object.move_robot(direction="forwards")
        rate.sleep()

<p style="background:#3B8F10;color:white;" id="prg-2-1">**END Python File: cmd_vel_publisher.py** </p>

<table style="float:left;background: #407EAF">
<tr>
<th>
<p class="transparent">3. Create Two Topic Subscribers that extract the data that you need from the Odometry and the Laser.</p>
</th>
</tr>
</table>

<p style="background:#3B8F10;color:white;" id="prg-2-1">**Python File: laser_sub.py** </p>

In [ ]:
#! /usr/bin/env python

import rospy
import time
from sensor_msgs.msg import LaserScan

class LaserTopicReader(object):
    def __init__(self, topic_name = '/kobuki/laser/scan'):
        self._topic_name = topic_name
        self._sub = rospy.Subscriber(self._topic_name, LaserScan, self.topic_callback)
        self._laserdata = LaserScan()
        self._front = 0.0
        self._right = 0.0
        self._left = 0.0
    
    def topic_callback(self, msg):
        self._laserdata = msg
        rospy.logdebug(self._laserdata)
    
    def get_laserdata(self):
        """
        Returns the newest odom data

        std_msgs/Header header
          uint32 seq
          time stamp
          string frame_id
        float32 angle_min
        float32 angle_max
        float32 angle_increment
        float32 time_increment
        float32 scan_time
        float32 range_min
        float32 range_max
        float32[] ranges
        float32[] intensities                                                                                                               
        
        """
        return self._laserdata    
    
if __name__ == "__main__":
    rospy.init_node('laser_topic_subscriber', log_level=rospy.INFO)
    laser_reader_object = LaserTopicReader()
    time.sleep(2)
    rate = rospy.Rate(0.5)
    
    ctrl_c = False
    def shutdownhook():
        # works better than the rospy.is_shut_down()
        global ctrl_c
        print "shutdown time!"
        ctrl_c = True

    rospy.on_shutdown(shutdownhook)
    
    while not ctrl_c:
        data = laser_reader_object.get_laserdata()
        rospy.loginfo(data)
        rate.sleep()

<p style="background:#3B8F10;color:white;" id="prg-2-1">**END Python File: laser_sub.py** </p>

<p style="background:#3B8F10;color:white;" id="prg-2-1">**Python File: odom_topic_subscriber.py** </p>

In [ ]:
#! /usr/bin/env python

import rospy
from nav_msgs.msg import Odometry

class OdomTopicReader(object):
    def __init__(self, topic_name = '/odom'):
        self._topic_name = topic_name
        self._sub = rospy.Subscriber(self._topic_name, Odometry, self.topic_callback)
        self._odomdata = Odometry()
    
    def topic_callback(self, msg):
        self._odomdata = msg
        rospy.logdebug(self._odomdata)
    
    def get_odomdata(self):
        """
        Returns the newest odom data

        std_msgs/Header header                                                                                                                 
          uint32 seq                                                                                                                           
          time stamp                                                                                                                           
          string frame_id                                                                                                                      
        string child_frame_id                                                                                                                  
        geometry_msgs/PoseWithCovariance pose                                                                                                  
          geometry_msgs/Pose pose                                                                                                              
            geometry_msgs/Point position                                                                                                       
              float64 x                                                                                                                        
              float64 y                                                                                                                        
              float64 z                                                                                                                        
            geometry_msgs/Quaternion orientation                                                                                               
              float64 x                                                                                                                        
              float64 y                                                                                                                        
              float64 z                                                                                                                        
              float64 w                                                                                                                        
          float64[36] covariance                                                                                                               
        geometry_msgs/TwistWithCovariance twist                                                                                                
          geometry_msgs/Twist twist                                                                                                            
            geometry_msgs/Vector3 linear                                                                                                       
              float64 x                                                                                                                        
              float64 y                                                                                                                        
              float64 z                                                                                                                        
            geometry_msgs/Vector3 angular                                                                                                      
              float64 x                                                                                                                        
              float64 y                                                                                                                        
              float64 z                                                                                                                        
          float64[36] covariance                                                                                                               
        
        """
        return self._odomdata
    
if __name__ == "__main__":
    rospy.init_node('odom_topic_subscriber', log_level=rospy.INFO)
    odom_reader_object = OdomTopicReader()
    rospy.loginfo(odom_reader_object.get_odomdata())
    rate = rospy.Rate(0.5)
    
    ctrl_c = False
    def shutdownhook():
        # works better than the rospy.is_shut_down()
        global ctrl_c
        print "shutdown time!"
        ctrl_c = True

    rospy.on_shutdown(shutdownhook)
    
    while not ctrl_c:
        data = odom_reader_object.get_odomdata()
        rospy.loginfo(data)
        rate.sleep()

<p style="background:#3B8F10;color:white;" id="prg-2-1">**END Python File: odom_topic_subscriber.py** </p>

## Solution Step 2: Use Topics through Services <p id="SolutionStep2"></p>

<table style="float:left;background: #407EAF">
<tr>
<th>
<p class="transparent">2. Modify the /kobuki/laser/scan topic subscriber</p>
</th>
</tr>
</table>

<p style="background:#3B8F10;color:white;" id="prg-2-1">**Python File: laser_sub.py** </p>

In [ ]:
#! /usr/bin/env python

import rospy
import time
from sensor_msgs.msg import LaserScan

class LaserTopicReader(object):
    def __init__(self, topic_name = '/kobuki/laser/scan'):
        self._topic_name = topic_name
        self._sub = rospy.Subscriber(self._topic_name, LaserScan, self.topic_callback)
        self._laserdata = LaserScan()
        self._front = 0.0
        self._right = 0.0
        self._left = 0.0
    
    def topic_callback(self, msg):
        self._laserdata = msg
        rospy.logdebug(self._laserdata)
    
    def get_laserdata(self):
        """
        Returns the newest odom data

        std_msgs/Header header
          uint32 seq
          time stamp
          string frame_id
        float32 angle_min
        float32 angle_max
        float32 angle_increment
        float32 time_increment
        float32 scan_time
        float32 range_min
        float32 range_max
        float32[] ranges
        float32[] intensities                                                                                                               
        
        """
        return self._laserdata
        
    def crash_detector(self):
        
        self._front = self._laserdata.ranges[360]
        self._right = self._laserdata.ranges[0]
        self._left = self._laserdata.ranges[719]
        rospy.loginfo("Front Distance == "+str(self._front))
        rospy.loginfo("Left Distance == "+str(self._left))
        rospy.loginfo("Right Distance == "+str(self._right))
        
        
        return self.convert_to_dict()
        
        
    def convert_to_dict(self):
        """
        Converts the fiven message to a dictionary telling in which direction there is a detection
        """
        detect_dict = {}
        # We consider that when there is a big Z axis component there has been a very big front crash
        detection_dict = {"front":self._front,
                          "left":self._left,
                          "right":self._right}
        return detection_dict
        
    
if __name__ == "__main__":
    rospy.init_node('laser_topic_subscriber', log_level=rospy.INFO)
    laser_reader_object = LaserTopicReader()
    time.sleep(2)
    rate = rospy.Rate(0.5)
    
    ctrl_c = False
    def shutdownhook():
        # works better than the rospy.is_shut_down()
        global ctrl_c
        print "shutdown time!"
        ctrl_c = True

    rospy.on_shutdown(shutdownhook)
    
    while not ctrl_c:
        data = laser_reader_object.get_laserdata()
        laser_reader_object.crash_detector()
        rate.sleep()

<p style="background:#3B8F10;color:white;" id="prg-2-1">**END Python File: laser_sub.py** </p>

<table style="float:left;background: #407EAF">
<tr>
<th>
<p class="transparent">3. Create the Server and client that tell you if there is a potential crash, and in what direction to move</p>
</th>
</tr>
</table>

<p style="background:#3B8F10;color:white;" id="prg-2-1">**Python File: direction_service_server.py** </p>

In [ ]:
#! /usr/bin/env python

import rospy
from std_srvs.srv import Trigger, TriggerResponse
from laser_sub import LaserTopicReader
import time


class CrashDirectionService(object):
    def __init__(self, srv_name='/crash_direction_service'):
        self._srv_name = srv_name
        self._laser_reader_object = LaserTopicReader()
        self.detection_dict = {"front":0.0, "left":0.0, "right":0.0}
        self._my_service = rospy.Service(self._srv_name, Trigger , self.srv_callback)

    def srv_callback(self, request):
        self.detection_dict = self._laser_reader_object.crash_detector()
        
        message = self.direction_to_move()
        
        rospy.logdebug("DIRECTION ==> "+message)
        
        response = TriggerResponse()
        """
        ---                                                                                                 
        bool success   # indicate if crashed                                       
        string message # Direction
        """
        response.success = self.potential_crash()
        response.message = message
        
        return response

    
    def potential_crash(self):
        
        if self.detection_dict["front"] < 0.8:
            return True
        else:
            return False
    
    def direction_to_move(self):

        if self.detection_dict["right"] > self.detection_dict["left"]:
            message = "right"
        
        else:
            message = "left"
        
        return message

if __name__ == "__main__":
    rospy.init_node('crash_direction_service_server', log_level=rospy.INFO) 
    dir_serv_object = CrashDirectionService()
    rospy.spin() # mantain the service open.

<p style="background:#3B8F10;color:white;" id="prg-2-1">**END Python File: direction_service_server.py** </p>

<p style="background:#3B8F10;color:white;" id="prg-2-1">**Python File: direction_service_client.py** </p>

In [ ]:
#! /usr/bin/env python

import rospy
from std_srvs.srv import Trigger, TriggerRequest
import sys 

rospy.init_node('crash_direction_service_client') # initialise a ROS node with the name service_client
service_name = "/crash_direction_service"
rospy.wait_for_service(service_name) # wait for the service client /gazebo/delete_model to be running
direction_service = rospy.ServiceProxy(service_name, Trigger) # create the connection to the service
request_object = TriggerRequest()

rate = rospy.Rate(5)

ctrl_c = False
def shutdownhook():
    # works better than the rospy.is_shut_down()
    global ctrl_c
    print "shutdown time!"
    ctrl_c = True

rospy.on_shutdown(shutdownhook)

while not ctrl_c:
    result = direction_service(request_object) # send through the connection the name of the object to be deleted by the service
    """
    ---                             
    bool success   # indicate succes
    string message # informational, 
    """
    if result.success:
        rospy.logwarn("Success =="+str(result.success)) # print the result given by the service called
        rospy.logwarn("Direction To Go=="+str(result.message)) # print the result given by the service called
    else:
        rospy.loginfo("Success =="+str(result.success)) # print the result given by the service called
        rospy.loginfo("Direction To Go=="+str(result.message)) # print the result given by the service called
    rate.sleep()

<p style="background:#3B8F10;color:white;" id="prg-2-1">**END Python File: direction_service_client.py** </p>

## Solution Step 3: Use Topics through Actions <p id="SolutionStep3"></p>

<table style="float:left;background: #407EAF">
<tr>
<th>
<p class="transparent">2. Create the action server, the client, and the program to compute out of the maze.</p>
</th>
</tr>
</table>

<p style="background:#3B8F10;color:white;" id="prg-2-1">**Python File: record_odom_action_server.py** </p>

In [ ]:
#! /usr/bin/env python

import rospy
from std_srvs.srv import Trigger, TriggerRequest
import sys 

rospy.init_node('crash_direction_service_client') # initialise a ROS node with the name service_client
service_name = "/crash_direction_service"
rospy.wait_for_service(service_name) # wait for the service client /gazebo/delete_model to be running
direction_service = rospy.ServiceProxy(service_name, Trigger) # create the connection to the service
request_object = TriggerRequest()

rate = rospy.Rate(5)

ctrl_c = False
def shutdownhook():
    # works better than the rospy.is_shut_down()
    global ctrl_c
    print "shutdown time!"
    ctrl_c = True

rospy.on_shutdown(shutdownhook)

while not ctrl_c:
    result = direction_service(request_object) # send through the connection the name of the object to be deleted by the service
    """
    ---                             
    bool success   # indicate succes
    string message # informational, 
    """
    if result.success:
        rospy.logwarn("Success =="+str(result.success)) # print the result given by the service called
        rospy.logwarn("Direction To Go=="+str(result.message)) # print the result given by the service called
    else:
        rospy.loginfo("Success =="+str(result.success)) # print the result given by the service called
        rospy.loginfo("Direction To Go=="+str(result.message)) # print the result given by the service called
    rate.sleep()

<p style="background:#3B8F10;color:white;" id="prg-2-1">**END Python File: record_odom_action_server.py** </p>

<p style="background:#3B8F10;color:white;" id="prg-2-1">**Python File: record_odom_action_client.py** </p>

In [ ]:
#! /usr/bin/env python

import rospy
from std_srvs.srv import Trigger, TriggerRequest
import sys 

rospy.init_node('crash_direction_service_client') # initialise a ROS node with the name service_client
service_name = "/crash_direction_service"
rospy.wait_for_service(service_name) # wait for the service client /gazebo/delete_model to be running
direction_service = rospy.ServiceProxy(service_name, Trigger) # create the connection to the service
request_object = TriggerRequest()

rate = rospy.Rate(5)

ctrl_c = False
def shutdownhook():
    # works better than the rospy.is_shut_down()
    global ctrl_c
    print "shutdown time!"
    ctrl_c = True

rospy.on_shutdown(shutdownhook)

while not ctrl_c:
    result = direction_service(request_object) # send through the connection the name of the object to be deleted by the service
    """
    ---                             
    bool success   # indicate succes
    string message # informational, 
    """
    if result.success:
        rospy.logwarn("Success =="+str(result.success)) # print the result given by the service called
        rospy.logwarn("Direction To Go=="+str(result.message)) # print the result given by the service called
    else:
        rospy.loginfo("Success =="+str(result.success)) # print the result given by the service called
        rospy.loginfo("Direction To Go=="+str(result.message)) # print the result given by the service called
    rate.sleep()

<p style="background:#3B8F10;color:white;" id="prg-2-1">**END Python File: record_odom_action_client.py** </p>

<p style="background:#3B8F10;color:white;" id="prg-2-1">**Python File: odometry_analysis.py** </p>

In [ ]:
#! /usr/bin/env python

import rospy
from nav_msgs.msg import Odometry
from geometry_msgs.msg import Vector3
import math 


class OdometryAnalysis(object):
    def __init__(self):
        pass
    
    def get_distance_moved(self, odmetry_data_list):
        
        distance = None
        
        if len(odmetry_data_list) >= 2 :
            start_odom = odmetry_data_list[0]
            end_odom = odmetry_data_list[len(odmetry_data_list)-1]
            
            start_position = start_odom.pose.pose.position
            end_position = end_odom.pose.pose.position
            
            rospy.loginfo("start_position ==>"+str(start_position))
            rospy.loginfo("end_position ==>"+str(end_position))
            
            
            distance_vector = self.create_vector(start_position, end_position)
            rospy.loginfo("Distance Vector ==>"+str(distance_vector))
            
            distance = abs(distance_vector.y)
            rospy.loginfo("Distance ==>"+str(distance))
        
        else:
            rospy.logerr("Odom array doesnt have the minimum number of elements = "+str(len(odmetry_data_list)))
        
        return distance
        
    def create_vector(self, p1, p2):
        
        distance_vector = Vector3()
        distance_vector.x = p2.x - p1.x
        distance_vector.y = p2.y - p1.y
        distance_vector.z = p2.z - p1.z
        
        return distance_vector
    


def check_if_out_maze(goal_distance, odom_result_array):
    odom_analysis_object = OdometryAnalysis()
    distance = odom_analysis_object.get_distance_moved(odom_result_array)
    rospy.loginfo("Distance Moved="+str(distance))
    
    return distance > goal_distance

<p style="background:#3B8F10;color:white;" id="prg-2-1">**END Python File: odometry_analysis.py** </p>

## Solution Step 4: Create a Main Program to Manage Everything <p id="SolutionStep4"></p>

<table style="float:left;background: #407EAF">
<tr>
<th>
<p class="transparent">2. Create a launch file that launches the Action Server, the Service Server, and the main program.</p>
</th>
</tr>
</table>

<p style="background:#3B8F10;color:white;" id="prg-2-1">**Launch File: main_launch.launch** </p>

In [ ]:
<launch>
    
    <node pkg ="turtle_project_sol"
        type="record_odom_action_server.py"
        name="record_odom_action_server_node"
        output="screen">
  </node>
  
  <node pkg ="turtle_project_sol"
        type="direction_service_server.py"
        name="crash_direction_service_server"
        output="screen">
  </node>

  <node pkg ="turtle_project_sol"
        type="main_program.py"
        name="turtlebot_main_node"
        output="screen">
  </node>
    
</launch>

<p style="background:#3B8F10;color:white;" id="prg-2-1">**END Launch File: main_launch.launch** </p>

<table style="float:left;background: #407EAF">
<tr>
<th>
<p class="transparent">3. Create the main program.</p>
</th>
</tr>
</table>

<p style="background:#3B8F10;color:white;" id="prg-2-1">**Python File: main_program.py** </p>

In [ ]:
#! /usr/bin/env python

import rospy
import actionlib
import time
from std_srvs.srv import Trigger, TriggerRequest
from turtle_project_sol.msg import record_odomGoal, record_odomFeedback, record_odomResult, record_odomAction
from cmd_vel_publisher import CmdVelPub
from odometry_analysis import OdometryAnalysis
from odometry_analysis import check_if_out_maze

class ControlTurtlebot(object):
    def __init__(self, goal_distance):
        self._goal_distance = goal_distance
        self.init_direction_service_client()
        self.init_rec_odom_action_client()
        self.init_move_turtlebot_publisher()
        
    def init_direction_service_client(self, service_name = "/crash_direction_service"):
        rospy.loginfo('Waiting for Service Server')
        rospy.wait_for_service(service_name) # wait for the service client /gazebo/delete_model to be running
        rospy.loginfo('Service Server Found...')
        self._direction_service = rospy.ServiceProxy(service_name, Trigger) # create the connection to the service
        self._request_object = TriggerRequest()
        
    def make_direction_request(self):
        
        result = self._direction_service(self._request_object) # send the name of the object to be deleted by the service through the connection
        return result
    
    def init_rec_odom_action_client(self):
        self._rec_odom_action_client = actionlib.SimpleActionClient('/rec_odom_as', record_odomAction)
        # waits until the action server is up and running
        rospy.loginfo('Waiting for action Server')
        self._rec_odom_action_client.wait_for_server()
        rospy.loginfo('Action Server Found...')
        self._rec_odom_action_goal = record_odomGoal()
    
    def send_goal_to_rec_odom_action_server(self):
        self._rec_odom_action_client.send_goal(self._rec_odom_action_goal, feedback_cb=self.rec_odom_feedback_callback)
        
    def rec_odom_feedback_callback(self,feedback):
        rospy.loginfo("Rec Odom Feedback feedback ==>"+str(feedback))
    
    def rec_odom_finished(self):
        
        has_finished = ( self._rec_odom_action_client.get_state() >= 2 )
        
        return has_finished
    
    def get_result_rec_odom(self):
        return self._rec_odom_action_client.get_result()
        
    def init_move_turtlebot_publisher(self):
        self._cmdvelpub_object = CmdVelPub()

    def move_turtlebot(self, direction):
        self._cmdvelpub_object.move_robot(direction)

    def got_out_maze(self, odom_result_array):
        return check_if_out_maze(self._goal_distance, odom_result_array)

rospy.init_node("turtlebot_main_node", log_level=rospy.INFO)
control_turtlebot_object = ControlTurtlebot(goal_distance=8.77)
rate = rospy.Rate(10)

control_turtlebot_object.send_goal_to_rec_odom_action_server()
i = 0

while not control_turtlebot_object.rec_odom_finished():

    direction_to_go = control_turtlebot_object.make_direction_request()
    
    if direction_to_go.success is False:
        control_turtlebot_object.move_turtlebot("forwards")
    else:
        control_turtlebot_object.move_turtlebot("stop")
        time.sleep(2)
        control_turtlebot_object.move_turtlebot(direction_to_go.message)
        if i==0:
            time.sleep(6.1)
            
        elif i==1:
            time.sleep(6.15)
            
        elif i==2:
            time.sleep(7)
            
        else:
            time.sleep(6.2)
        
        i=i+1    
        control_turtlebot_object.move_turtlebot("stop")
        time.sleep(2)
        
    rate.sleep()
    


odom_result = control_turtlebot_object.get_result_rec_odom()
odom_result_array = odom_result.result_odom_array

if control_turtlebot_object.got_out_maze(odom_result_array):
    control_turtlebot_object.move_turtlebot("stop")
    time.sleep(2)
    rospy.loginfo("Out of Maze")
else:
    rospy.loginfo("In Maze")
    control_turtlebot_object.move_turtlebot("stop")
    time.sleep(2)

rospy.loginfo("Turtlebot Maze test Finished")

<p style="background:#3B8F10;color:white;" id="prg-2-1">**END Python File: main_program.py** </p>